# Uproot and Awkward Array tutorial

**July 6, 2021 at PyHEP**

<br><br><br><br><br>

## Z peak in PyROOT

Let's start with the most straightforward way to analyze HEP data in Python: "for" loops in PyROOT.

In [ ]:
import time
import numpy as np
import ROOT

canvas = ROOT.TCanvas()

In [ ]:
rootfile = ROOT.TFile.Open("data/HiggsZZ4mu.root")
roottree = rootfile.Get("Events")

In [ ]:
starttime = time.time()

roothist = ROOT.TH1D("roothist", "mass", 120, 0, 120)

for index, event in enumerate(roottree):
    # cuts are "if" statements
    if event.nMuon >= 2 and event.Muon_charge[0] + event.Muon_charge[1] == 0:
        mu1_pt = event.Muon_pt[0]
        mu2_pt = event.Muon_pt[1]
        mu1_eta = event.Muon_eta[0]
        mu2_eta = event.Muon_eta[1]
        mu1_phi = event.Muon_phi[0]
        mu2_phi = event.Muon_phi[1]

        # histograms are filled in the loop
        roothist.Fill(
            np.sqrt(2*mu1_pt*mu2_pt*(np.cosh(mu1_eta - mu2_eta) - np.cos(mu1_phi - mu2_phi)))
        )

pyroot_time = time.time() - starttime
print(f"total time: {pyroot_time} sec")

In [ ]:
roothist.Draw()
canvas.Draw()

<br><br><br><br><br>

## Z peak in C++ ROOT

It works, but 20 seconds is a long time to wait for 1/8 million dimuons.

The traditional way to speed things up is to translate it into C++.

In [ ]:
ROOT.gInterpreter.Declare('''
void compute(TH1D& roothist, TTree& roottree) {
    UInt_t nMuon;
    float Muon_pt[50];
    float Muon_eta[50];
    float Muon_phi[50];
    int32_t Muon_charge[50];

    roottree.SetBranchStatus("*", 0);
    roottree.SetBranchStatus("nMuon", 1);
    roottree.SetBranchStatus("Muon_pt", 1);
    roottree.SetBranchStatus("Muon_eta", 1);
    roottree.SetBranchStatus("Muon_phi", 1);
    roottree.SetBranchStatus("Muon_charge", 1);

    roottree.SetBranchAddress("nMuon", &nMuon);
    roottree.SetBranchAddress("Muon_pt", Muon_pt);
    roottree.SetBranchAddress("Muon_eta", Muon_eta);
    roottree.SetBranchAddress("Muon_phi", Muon_phi);
    roottree.SetBranchAddress("Muon_charge", Muon_charge);

    for (int index = 0; index < 100000; index++) {
        roottree.GetEntry(index);
        if (nMuon >= 2 && Muon_charge[0] + Muon_charge[1] == 0) {
            float mu1_pt = Muon_pt[0];
            float mu2_pt = Muon_pt[1];
            float mu1_eta = Muon_eta[0];
            float mu2_eta = Muon_eta[1];
            float mu1_phi = Muon_phi[0];
            float mu2_phi = Muon_phi[1];
            roothist.Fill(
                sqrt(2*mu1_pt*mu2_pt*(cosh(mu1_eta - mu2_eta) - cos(mu1_phi - mu2_phi)))
            );
        }
    }
}
''')

ROOT lets you compile a C++ function and run it through PyROOT.

The important thing here is that the _loop over events_ is in the compiled code.

In [ ]:
starttime = time.time()

roothist2 = ROOT.TH1D("roothist2", "mass", 120, 0, 120)

ROOT.compute(roothist2, roottree)

cpproot_time = time.time() - starttime
print(f"total time: {cpproot_time} sec")

In [ ]:
pyroot_time / cpproot_time

In [ ]:
roothist2.Draw()
canvas.Draw()

<br><br><br><br><br>

## Z peak in RDataFrame

RDataFrame is the modern way to build workflows over HEP data in ROOT.

Each node in this pipeline is compiled in C++. The pipeline itself can be built in Python.

<img src="img/rdataframe-flow.svg" style="width: 800px">

In [ ]:
df = ROOT.RDataFrame("Events", "data/HiggsZZ4mu.root")

# Each node is connected to the previous, in a chain (which can split and recombine).
df_2mu = df.Filter("nMuon >= 2")
df_os = df_2mu.Filter("Muon_charge[0] + Muon_charge[1] == 0")

# This node is a big C++ block.
df_mass = df_os.Define("Dimuon_mass", '''
float mu1_pt = Muon_pt[0];
float mu2_pt = Muon_pt[1];
float mu1_eta = Muon_eta[0];
float mu2_eta = Muon_eta[1];
float mu1_phi = Muon_phi[0];
float mu2_phi = Muon_phi[1];
return sqrt(2*mu1_pt*mu2_pt*(cosh(mu1_eta - mu2_eta) - cos(mu1_phi - mu2_phi)));
''')

roothist3 = df_mass.Histo1D(("h3", "mass", 120, 0, 120), "Dimuon_mass")

The calculation doesn't actually start until you attempt to look at a result (convert to NumPy or draw a plot).

In [ ]:
starttime = time.time()

# This one is an endpoint (action).
array = df_mass.AsNumpy(["Dimuon_mass"])

rdfroot_time = time.time() - starttime
print(f"total time: {rdfroot_time} sec")

In [ ]:
pyroot_time / rdfroot_time

In [ ]:
array

In [ ]:
roothist3.Draw()
canvas.Draw()

<br><br><br><br><br>

## Uproot and Awkward Array

Uproot's approach is different: each Python command operates on a whole array at a time (like NumPy).

Loops over all events happen in compiled code, but you don't write that code—you combine primitives to build a workflow.

In [ ]:
import awkward as ak
import uproot
import hist

Open a file and look at its contents.

In [ ]:
events = uproot.open("data/HiggsZZ4mu.root:Events")
events.show()

Extract some branches as an "array of records" (renaming the fields).

In [ ]:
muons = events.arrays(
    ["pt", "eta", "phi", "charge"],
    aliases={"pt": "Muon_pt", "eta": "Muon_eta", "phi": "Muon_phi", "charge": "Muon_charge"}
)
muons

In [ ]:
muons.fields

A cut is an array of booleans, which we can construct as a formula.

In [ ]:
cut = (ak.num(muons.charge) >= 2) & (ak.sum(muons.charge[:, :2], axis=1) == 0)
cut

Applying a cut is a slice. In the same slice, we can pick the first (`0`) and second (`1`) muon in each event.

In [ ]:
mu1 = muons[cut, 0]
mu2 = muons[cut, 1]
mu1, mu2

Let's use the [hist](https://github.com/scikit-hep/hist#readme) library for histograms.

In [ ]:
h = hist.Hist.new.Reg(120, 0, 120, name="mass").Double()
h

In [ ]:
h.fill(np.sqrt(2*mu1.pt*mu2.pt*(np.cosh(mu1.eta - mu2.eta) - np.cos(mu1.phi - mu2.phi))))

And Matplotlib for plots.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
h.plot();

<br><br><br><br><br>

Now let's do the whole thing in one cell and time it.

In [ ]:
starttime = time.time()

# read data
muons = events.arrays(
    ["pt", "eta", "phi", "charge"],
    aliases={"pt": "Muon_pt", "eta": "Muon_eta", "phi": "Muon_phi", "charge": "Muon_charge"},
    array_cache=None,   # no cheating!
)

# compute
cut = (ak.num(muons.charge) >= 2) & (ak.sum(muons.charge[:, :2], axis=1) == 0)
mu1 = muons[cut, 0]
mu2 = muons[cut, 1]
h = hist.Hist.new.Reg(120, 0, 120, name="mass").Double()
h.fill(np.sqrt(2*mu1.pt*mu2.pt*(np.cosh(mu1.eta - mu2.eta) - np.cos(mu1.phi - mu2.phi))))

uproot_time = time.time() - starttime
print(f"total time: {uproot_time} sec")

In [ ]:
pyroot_time / uproot_time

It's in the same ballpark as C++. It can be 1.5× to 2× slower, but it's much closer to C++ on a log plot than it is to Python "for" loops.

<br><br><br><br><br>

## The Pythonic HEP ecosystem

Uproot is not a framework, it _only_ does ROOT I/O. Awkward Array handles array manipulation, hist does histograms, etc.

It is part of this complete breakfast:

<img src="img/logo-parade.svg" width="800px">

<br><br><br><br><br>

## Uproot

Uproot is an independent implementation of ROOT I/O that leverages standard Python libraries.

<img src="img/abstraction-layers.svg" width="800px">

<br><br><br><br><br>

The documentation is at [https://uproot.readthedocs.io/](https://uproot.readthedocs.io/).

<img src="img/uproot-documentation.png" width="800px"/>

<br><br><br><br><br>

## Navigating a ROOT file

<img src="img/terminology.svg" style="width: 800px">

In [ ]:
histograms = uproot.open("data/HiggsZZ4mu_histograms.root")
histograms

In [ ]:
histograms.file

In Uproot, a directory is a dict-like object with subscripting (square brackets), [keys](https://uproot.readthedocs.io/en/latest/uproot.reading.ReadOnlyDirectory.html#keys), [values](https://uproot.readthedocs.io/en/latest/uproot.reading.ReadOnlyDirectory.html#values), [items](https://uproot.readthedocs.io/en/latest/uproot.reading.ReadOnlyDirectory.html#items), and [classnames](https://uproot.readthedocs.io/en/latest/uproot.reading.ReadOnlyDirectory.html#classnames) methods.

In [ ]:
histograms["Z"]

In [ ]:
histograms["Z"]["4mu"]

Try `recursive`, `filter_name`, and `filter_classname` arguments.

In [ ]:
histograms.keys()

Most histograms and graphs can be converted to types in other Python libraries.

Try the `to_hist()` method on this one.

In [ ]:
histograms["Z/all/massZto2muon"]

<br><br><br>

**Three-minute exercise:** find the 2D histogram and plot it.

In [ ]:
histograms

<br><br><br><br><br>

## Can Uproot read my data?

<img src="img/can-uproot-read-it.svg" width="500px"/>

Here is a file with analysis-specific classes; Uproot _cannot_ have prior knowledge of these classes.

In [ ]:
icecube = uproot.open("data/icecube-supernovae.root")
icecube.keys()

In [ ]:
icecube.classname_of("config/detector")

It is possible to read an `I3Eval_t` object because ROOT stores the "how to read" instructions in the file (called "streamers").

In [ ]:
icecube.file.show_streamers("I3Eval_t")

In [ ]:
icecube["config/detector"]

In [ ]:
icecube["config/detector"].all_members

In [ ]:
icecube["config/detector"].member("ChannelIDMap")

Unlike histograms, these objects have no methods to help you unpack the data; it's all in [members](https://uproot.readthedocs.io/en/latest/uproot.model.Model.html#members), [all_members](https://uproot.readthedocs.io/en/latest/uproot.model.Model.html#all-members), [has_member](https://uproot.readthedocs.io/en/latest/uproot.model.Model.html#has-member), and [member](https://uproot.readthedocs.io/en/latest/uproot.model.Model.html#member).

<br><br><br><br><br>

## Navigating TTrees

In [ ]:
zmumu_file = uproot.open("data/Zmumu.root")
zmumu_file.classnames()

Often, the best thing to do first with an unfamiliar TTree is [show](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#show).

In [ ]:
zmumu = zmumu_file["events"]
zmumu.show()

Keep in mind that

   * TTrees are dict-like objects with subscripting (square brackets), [keys](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#keys), [values](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#values), [items](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#items), and [typenames](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#typenames) methods (like directories)
   * you can access all of the above data with methods: you don't have to parse the [show](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#show) string!

In [ ]:
zmumu.keys()

In [ ]:
zmumu.typenames()

In [ ]:
{name: branch.interpretation for name, branch in zmumu.items()}

<br><br><br><br><br>

## Reading TBranches into arrays

The basic method is to get a TBranch (with square brackets) and call [array](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.TBranch.html#array).

In [ ]:
zmumu["M"].array()

Some important parameters:

   * `entry_start`, `entry_stop` to limit how much you read (if it's big)
   * `library="np"` for NumPy arrays, `library="ak"` for Awkward Arrays, and `library="pd"` for Pandas (Series or DataFrame)

In [ ]:
zmumu["M"].array(entry_stop=5)

Get a NumPy (not Awkward) array:

In [ ]:
zmumu["M"].array(library="np")

Get a Pandas Series:

In [ ]:
zmumu["M"].array(library="pd")

<br><br><br><br><br>

## Reading many arrays at once

The [arrays](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.HasBranches.html#arrays) method retrieves many TBranches into a "group" of arrays.

What a "group" means depends on the library.

Awkward Arrays group data in records (substructure within the array).

In [ ]:
zmumu.arrays()

A "group" of NumPy arrays is a dict (unless you specify `how`):

In [ ]:
zmumu.arrays(library="np")

A "group" of Pandas Series is a DataFrame:

In [ ]:
zmumu.arrays(library="pd")

The first argument can be used to extract TBranches by name:

In [ ]:
zmumu.arrays(["px1", "py1", "px2", "py2"], library="pd")

But this argument actually takes arbitrary (Python) expressions.

In [ ]:
zmumu.arrays(["sqrt(px1**2 + py1**2)", "sqrt(px2**2 + py2**2)"], library="pd")

This is to support any [aliases](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#aliases) that might be in the [TTree](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html), but you can make up your own `aliases` on the spot.

In [ ]:
zmumu.arrays(
    ["pt1", "pt2"],
    {"pt1": "sqrt(px1**2 + py1**2)", "pt2": "sqrt(px2**2 + py2**2)"},
    library="pd",
)

The fact that these are interpreted as expressions has some "gotchas":

   * nested branches, paths with "`/`", _would be interpreted as division!_
   * wildcards, paths with "`*`", _would be interpreted as multiplication!_

For pattern-matching on TBranches, use the `filter_name`, `filter_typename`, and `filter_branch` arguments of [arrays](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.HasBranches.html#arrays).

In [ ]:
zmumu.arrays(filter_name="p[xyz]*", library="pd")

These filters have the same meaning as in [keys](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.HasBranches.html#keys) and [typenames](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.HasBranches.html#typenames), so you can test your filters without reading data.

In [ ]:
zmumu.keys(filter_name="p[xyz]*")

In [ ]:
zmumu.typenames(filter_name="p[xyz]*")

<br><br><br>

### Get arrays in manageable chunks

The [iterate](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.HasBranches.html#iterate) method is like [arrays](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.HasBranches.html#arrays), but it can be used in a loop over chunks of the array.

How large are the chunks? You should set that with `step_size`.

In [ ]:
for arrays in zmumu.iterate(step_size=300):
    print(repr(arrays))

In [ ]:
for arrays in zmumu.iterate(step_size="50 kB"):   # 50 kB is very small! for illustrative purposes only!
    print(repr(arrays))

<br><br><br>

### Collections of files (like TChain)

Each of the single-TTree array-reading methods has a corresponding multi-file function.

   * The equivalent of TTree [arrays](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#arrays) is [uproot.concatenate](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.concatenate.html). _(Reads everything at once: use this as a convenience on datasets you know are small!)_
   * The equivalent of TTree [iterate](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#iterate) is [uproot.iterate](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.iterate.html). _(This is the most useful one.)_
   * There's also an [uproot.lazy](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.lazy.html) _(More on this below.)_

Demo of scanning through a large (remote) file:

In [ ]:
import IPython
import matplotlib.pyplot as plt
import matplotlib.pylab

In [ ]:
h = hist.Hist.new.Reg(100, 0, 500, name="mass").Double()

for muons in uproot.iterate(
    # filename(s)
    ["root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root:Events"],

    # expressions
    ["pt", "eta", "phi", "charge"],
    aliases={"pt": "Muon_pt", "eta": "Muon_eta", "phi": "Muon_phi", "charge": "Muon_charge"},    

    # the all-important step_size!
    step_size="1 MB",
):
    # do everything you're going to do to this array
    cut = (ak.num(muons.charge) >= 2) & (ak.sum(muons.charge[:, :2], axis=1) == 0)
    mu1 = muons[cut, 0]
    mu2 = muons[cut, 1]

    # such as filling a histogram
    h.fill(np.sqrt(2*mu1.pt*mu2.pt*(np.cosh(mu1.eta - mu2.eta) - np.cos(mu1.phi - mu2.phi))))

    h.plot()
    plt.yscale("log")
    IPython.display.display(matplotlib.pylab.gcf())
    IPython.display.clear_output(wait=True)

    if h.counts().sum() > 300000:
        break

<br><br><br><br><br>

Lazy reading is like iterative reading, but it fetches the data only when needed.

In [ ]:
lazy = uproot.lazy(
    # filename(s)
    ["root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root:Events"],
    # step_size is still important
    step_size="1 MB",
)
lazy

In [ ]:
lazy.Muon_pt

In [ ]:
lazy.Muon_eta

**Important:** this is not the most efficient way to iterate through a file when you _know_ which TBranches you want to read.

It exists for interactive exploration.

<br><br><br><br><br>

## What if the data are not one-dimensional or rectilinear arrays?

Consider this ROOT file (simulation used in the Higgs discovery, converted to NanoAOD).

In [ ]:
events = uproot.open("data/HiggsZZ4mu.root:Events")

Some data, such as missing energy (MET), consist of a single value per collision event and can be represented by normal NumPy arrays.

In [ ]:
nonjagged = events["MET_pt"].array(entry_stop=20, library="np")
nonjagged

Normal NumPy slicing rules apply.

In [ ]:
nonjagged[:5]

<br><br><br><br><br>

Some data cannot. There's a different number of muons in each event, so we need variable-length nested lists to represent their transverse momenta ($p_T$).

In [ ]:
jagged_awkward = events["Muon_pt"].array(entry_stop=20)
jagged_awkward

In [ ]:
jagged_awkward.tolist()

It is _possible_ to read these data into NumPy, but with a considerable cost.

In [ ]:
jagged_numpy = events["Muon_pt"].array(entry_stop=20, library="np")
jagged_numpy

This is an _array of NumPy arrays_ (because you can put Python objects in NumPy arrays—not recommended).

You might want to consider the inner arrays to be a second dimension and use normal slicing rules:

In [ ]:
jagged_awkward[:, :1]

In [ ]:
jagged_numpy[:, :1]

In NumPy, you can't. NumPy doesn't know that all the contents of this array are arrays of the same type.

You know this and can write a loop in Python:

In [ ]:
np.array([x[:1] for x in jagged_numpy])

But this isn't recommended. It's non-idiomatic and slow.

<br><br><br><br><br>

Pandas can work with this "jagged" data through indexing.

In [ ]:
events.arrays(filter_name=["Muon_*"], library="pd")

But there are limitations. Try loading non-muon branches in the same DataFrame.

<br><br><br><br><br>

## Awkward Array

Awkward Array is a library for manipulating JSON-like data using NumPy-like idioms.

<img src="img/cartoon-schematic.svg" width="800px"/>

<br><br><br><br><br>

The documentation is at [https://awkward-array.org/](https://awkward-array.org/).

<img src="img/awkward-documentation.png" width="800px"/>

<br><br><br><br><br>

Consider this Parquet file of the same dataset. It can be read into an Awkward Array, just like the ROOT file.

In [ ]:
array = ak.from_parquet("data/HiggsZZ4mu.parquet")
array

In [ ]:
array.fields

In [ ]:
array[0].tolist()

In [ ]:
array.muons.pt

<br><br><br><br><br>

I've restructured it a little ("NanoEvents-style" vs "NanoAOD-style"), but we can easily make the data from ROOT look just like the data from Parquet.

In [ ]:
nanoaod_style = events.arrays(filter_name="Muon_*")
nanoaod_style.type

In [ ]:
array.muons.type

In [ ]:
nanoevents_style = ak.zip({
    "pt": nanoaod_style.Muon_pt,
    "eta": nanoaod_style.Muon_eta,
    "phi": nanoaod_style.Muon_phi,
    "mass": nanoaod_style.Muon_mass,
    "charge": nanoaod_style.Muon_charge,
})
nanoevents_style.type

<br><br><br><br><br>

In general, Awkward Array data types "`T`" can be:

   * numbers, booleans, date-times, etc.
   * variable-length and fixed-length lists of `T`
   * records with named or unnamed (tuple) fields of type `T1`, `T2`, ...
   * missing values: `T` _or_ `None`
   * heterogeneous types: `T1` _or_ `T2` _or_ ...

<br><br><br><br><br>

## Changing structure

Although the `tolist()` form of the data looks like JSON objects, the data are actually in a very fluid "columnar" form.

You can, for instance, turn an array from entry-per-event into entry-per-lumiblock by increasing the nesting by one.

<img src="img/events-to-lumis.svg" width="800px"/>

In [ ]:
array.luminosityBlock

In [ ]:
lumilengths = ak.run_lengths(array.luminosityBlock)
lumilengths

In [ ]:
array_by_lumi = ak.unflatten(array, lumilengths, axis=0)
array_by_lumi

This is an array of lists (luminosity blocks) of records (collision events) containing lists of records (muons, generator-level particles, etc.).

In [ ]:
array_by_lumi.luminosityBlock[0]

In [ ]:
array.type

In [ ]:
array_by_lumi.type

So, for example, you can add up $p_T$ values along any of three dimensions.

In [ ]:
ak.sum(array_by_lumi.muons.pt, axis=-1)

Summing (or "reducing" in general) is well-defined for irregular data shapes, depending on a choice of alignment. We left-align.

<img src="img/example-reduction.svg" style="width: 800px">

<br><br><br><br><br>

## Practical analysis

To get beyond theory, let's do some realistic things with the data.

In [ ]:
array.muons

A cut is an array of booleans.

In [ ]:
cut = ak.num(array.muons) >= 2
cut

In [ ]:
array.muons[cut]

Notice that there are now fewer events than there had been before.

Sometimes, that's a problem for composing cuts—having to know which cuts have been applied to which arrays of booleans.

We can avoid that problem by masking: rejected events are replaced by "`None`", rather than removed.

In [ ]:
array.muons.mask[cut]

In [ ]:
selected_muons = array.muons[cut]

selected_muons.charge[:, 0] + selected_muons.charge[:, 1] == 0

The `[:, 0]`, `[:, 1]` syntax assumes that a first and second muon exists (it does, because of the selection) and ignores all others.

In you want to consider combinations of all good particles in each event, so there are functions for constructing that.

<table style="margin-left: 0px">
    <tr style="background: white"><td style="font-size: 1.75em; font-weight: bold; text-align: center"><a href="https://awkward-array.readthedocs.io/en/latest/_auto/ak.cartesian.html">ak.cartesian</a></td><td style="font-size: 1.75em; font-weight: bold; text-align: center"><a href="https://awkward-array.readthedocs.io/en/latest/_auto/ak.combinations.html">ak.combinations</a></td></tr>
    <tr style="background: white"><td><img src="img/cartoon-cartesian.svg"></td><td><img src="img/cartoon-combinations.svg"></td></tr>
</table>

In [ ]:
ak.combinations(array.muons, 2).type

In [ ]:
mu1, mu2 = ak.unzip(ak.combinations(array.muons, 2))
mu1, mu2

These are not just the first two muons in each event: they are all combinations of two (without duplication).

In [ ]:
h = hist.Hist.new.Reg(120, 0, 120, name="mass").Double()
h.fill(ak.flatten(
    np.sqrt(2*mu1.pt*mu2.pt*(np.cosh(mu1.eta - mu2.eta) - np.cos(mu1.phi - mu2.phi)))
))

<br><br><br><br><br>

## The Vector library

Vector is a library for doing coordinate transformations and 2D, 3D, and 4D (Lorentz) calculations on _arrays_ of vectors.

Awkward Array is one of its backends.

In [ ]:
import vector
vector.register_awkward()

The above registers a suite of "[behaviors](https://awkward-array.readthedocs.io/en/latest/ak.behavior.html)" for Awkward Array, so that any records named `"Momentum4D"` now have Lorentz vector methods, as though they were members of a Lorentz vector class.

In [ ]:
muons = ak.with_name(array.muons, "Momentum4D")
muons

In [ ]:
muons[1, 0], muons[1, 1]

In [ ]:
muons[1, 0].cross(muons[1, 1])

But _arrays of vectors_ also have these methods, and they apply an array at a time.

In [ ]:
mu1, mu2 = ak.unzip(ak.combinations(muons, 2))
mu1, mu2

In [ ]:
mu1.cross(mu2)

In [ ]:
(mu1 + mu2).mass

In [ ]:
h = hist.Hist.new.Reg(120, 0, 120, name="mass").Double()
h.fill(ak.flatten((mu1 + mu2).mass))

<br><br><br><br><br>

## Example: reconstructed/generator-level matching with ΔR

In [ ]:
gen = ak.with_name(array.gen, "Momentum3D")
gen

In [ ]:
gen.fields

First, make all reco-gen pairs. (`nested=True` puts all pairs with a given reco muon in a new nested list.)

In [ ]:
reco_gen = ak.cartesian({"muon": muons, "gen": gen}, nested=True)
reco_gen

Break them into two arrays to make them easier to work with.

In [ ]:
mu, g = ak.unzip(reco_gen)
mu, g

Now we can compute an array of ΔR values.

In [ ]:
mu.deltaR(g)

In [ ]:
hist.Hist.new.Reg(100, 0, 5).Double().fill(
    ak.flatten(mu.deltaR(g), axis=None)
)

Some are very close to zero, some not.

How about if we look only at generator-level muons? Only non-muons?

In [ ]:
hist.Hist.new.Reg(100, 0, 5).Double().fill(
    ak.flatten(mu.deltaR(g)[abs(g.pdgId) == 13], axis=None)
)

What we want is not ΔR for _all_ reco-gen pairs, but the minimum ΔR for each reco muon.

[ak.min](https://awkward-array.readthedocs.io/en/latest/_auto/ak.min.html) is a reducer, removing that extra layer of nested list we made with `nested=True`.

In [ ]:
mu.deltaR(g)

In [ ]:
ak.min(mu.deltaR(g), axis=-1)

Zoom into small ranges of ΔR.

In [ ]:
hist.Hist.new.Reg(100, 0, 5).Double().fill(
    ak.flatten(ak.min(mu.deltaR(g), axis=-1), axis=None)
)

Instead of just plotting the minimum, let's get the [ak.argmin](https://awkward-array.readthedocs.io/en/latest/_auto/ak.argmin.html), the index position of the best ΔR.

(`keepdims=True` keeps the reducer from removing a dimension, which we'll need for the slice in the next step. The best indexes are in length-1 lists.)

In [ ]:
best = ak.argmin(mu.deltaR(g), axis=-1, keepdims=True)
best

This slice picks out the reco-gen pairs with minimal ΔR.

In [ ]:
reco_gen[best]

And there you have it: an array of `{muon: ABC, gen: XYZ}` pairs representing the best match for each reco muon.

In [ ]:
ak.flatten(reco_gen[best], axis=-1)[:4].tolist()

<br><br><br><br><br>

## Numba: a just-in-time compiler for Python

It's possible to do complex combinatorics with array-at-a-time functions, but nested "for" loops would often be easier.

Nested "for" loops can be fast if they're compiled.

[Numba](https://numba.pydata.org/) compiles Python.

In [ ]:
import numba as nb

Remember how long it takes to run a loop in Python?

In [ ]:
starttime = time.time()

sumpt = np.zeros(len(array), np.float64)
for i, event in enumerate(array):
    for muon in event.muons:
        sumpt[i] += muon.pt

python_time = time.time() - starttime
print(f"total time: {python_time} sec")

The same loop, in a function preceded by `@nb.jit`, is compiled by Numba when you first call it.

In [ ]:
@nb.jit
def calculate_sumpt(array):
    out = np.zeros(len(array), np.float64)
    for i, event in enumerate(array):
        for muon in event.muons:
            out[i] += muon.pt
    return out

In [ ]:
calculate_sumpt(array)

In [ ]:
starttime = time.time()

sumpt = calculate_sumpt(array)

numba_time = time.time() - starttime
print(f"total time: {numba_time} sec")

In [ ]:
python_time / numba_time

In many cases, Numba is _faster_ than the corresponding array-at-a-time function.

In [ ]:
starttime = time.time()

sumpt = ak.sum(array, axis=-1)

awkward_time = time.time() - starttime
print(f"total time: {awkward_time} sec")

In [ ]:
python_time / awkward_time

**Conclusion:** use array-at-a-time functions when you're working interactively or it's the most concise/easy-to-understand way to write an expression.

Use Numba when you need extreme speed or "for" loops are the most concise/easy-to-understand way to write it.

Convoluted code, just for the sake of using array-at-a-time functions, is not helping anybody!

<br><br><br><br><br>

## Limitations of Numba

Maybe this sounds too good to be true: "Python is slow, but put `@nb.jit` on each function and it will be fast."

The truth is that Numba only works on a _subset_ of Python. It replaces Python code with statically typed, compiled code, and Python is too dynamic of a language for that to always be possible. The Numba team keeps a list of [supported Python language features](https://numba.pydata.org/numba-doc/dev/reference/pysupported.html) and [supported NumPy functions](https://numba.pydata.org/numba-doc/dev/reference/numpysupported.html). Numba also only recognizes libraries that have been explicitly extended to work with it. Awkward Array and Vector have been extended; hist will be.

When it fails, the error messages can be hard to understand. Hint: start with a small, do-nothing function and gradually fold in the features you want, to better know which part is causing the error. See also [my tutorial on Numba](https://youtu.be/X_BJrmofRWQ).

If you're willing to learn a new language, [Julia](https://julialang.org/) is designed from the ground up as a just-in-time compilable language. See the session on Friday.

<img src="img/indico-julia.png" width="800px"/>

<br><br><br><br><br>

## Reconstructed/generator-level matching in Numba

Before we repeat the reco-gen matching exercise in Numba, let's build a simple Awkward Array output from a Numba-compiled function using [ak.ArrayBuilder](https://awkward-array.readthedocs.io/en/latest/_auto/ak.ArrayBuilder.html).

In [ ]:
@nb.jit
def build_nested(array, builder):
    for event in array:
        builder.begin_list()
        
        for muon in event.muons:
            builder.append(muon.pt)
        
        builder.end_list()
    
    return builder

build_nested(array, ak.ArrayBuilder()).snapshot()

It's the same as `array.muons.pt`, so this is definitely an example where the array-at-a-time function is simpler.

In [ ]:
array.muons.pt

Reco-gen matching, however, is simpler as a nested "for" loop.

Note that we don't have to output the fully formed array; it is enough to use Numba to make the index that we slice arrays with outside of the Numba-compiled function.

In [ ]:
@nb.jit
def matching(array_muons, array_gen, builder):
    for muons_event, gen_event in zip(array_muons, array_gen):
        builder.begin_list()

        for muon in muons_event:
            best_i = -1
            best_dr = -1.0
            for i, gen in enumerate(gen_event):
                dr = muon.deltaR(gen)
                if best_i < 0 or dr < best_dr:
                    best_i = i
                    best_dr = dr

            if best_i < 0:
                builder.append(None)
            else:
                builder.append(best_i)

        builder.end_list()

    return builder

index_of_best = matching(muons, gen, ak.ArrayBuilder()).snapshot()
index_of_best

This index picks the best generator-level particle for each reconstructed muon.

In [ ]:
gen_match = gen[index_of_best]
gen_match

In [ ]:
ak.num(gen), ak.num(muons), ak.num(gen_match)

So building the reco-gen pairs is just an [ak.zip](https://awkward-array.readthedocs.io/en/latest/_auto/ak.zip.html).

_This_ part would be harder in Numba. Use the best tool for each job.

In [ ]:
ak.zip({"muons": muons, "gen": gen_match})

<br><br><br><br><br>

## Example: identifying Z bosons in H → ZZ → 4μ

The Higgs boson decays to an "on-shell" Z boson (with a mass near 91 GeV) and an "off-shell" Z boson (much lower mass).

In a real analysis, it is necessary to know which is which, because different quality cuts are applied. Given only the four muons, finding the right pair of pairs is a combinatorics problem.

This example solves that problem using only array-at-a-time functions.

In [ ]:
four_muons = muons[(ak.num(muons) == 4) & (ak.sum(muons.charge, axis=-1) == 0)]
four_muons

General strategy: identify qualitatively distinct collections as separate named arrays.

The names will help you in thinking about the problem.

In [ ]:
mu_plus = four_muons[four_muons.charge > 0]
mu_minus = four_muons[four_muons.charge < 0]
mu_plus, mu_minus

By construction (the cut defining `four_muons`), all lists in `mu_plus` and `mu_minus` have exactly two items each.

In [ ]:
ak.num(mu_plus), ak.num(mu_minus)

You can check that explicitly to increase confidence and (if necessary) debug.

In [ ]:
ak.all(ak.num(mu_plus) == 2), ak.all(ak.num(mu_minus) == 2)

Knowing this (and the fact that 2 is not a large number), we can name each of these to further simplify our structures.

In [ ]:
mu_plus_0 = mu_plus[:, 0]
mu_plus_1 = mu_plus[:, 1]
mu_minus_0 = mu_minus[:, 0]
mu_minus_1 = mu_minus[:, 1]

mu_plus_0, mu_plus_1, mu_minus_0, mu_minus_1

Now, we _could_ do combinatorics using [ak.combinations](https://awkward-array.readthedocs.io/en/latest/_auto/ak.combinations.html), but with such a small number of known combinations, do it with explicitly named arrays. The structures will be simpler and the names will help you.

For each event, either `z00` and `z11` will be valid or `z01` and `z10` will be valid. The names help you remember this constraint to avoid using the same particle in multiple decays.

In [ ]:
z00 = mu_plus_0 + mu_minus_0
z11 = mu_plus_1 + mu_minus_1

z01 = mu_plus_0 + mu_minus_1
z10 = mu_plus_1 + mu_minus_0

As an aside, instead of hard-coding the Z mass, get it from the [particle](https://github.com/scikit-hep/particle#readme) package, which is like a Pythonic PDG.

In [ ]:
import particle

In [ ]:
particle.Particle.from_string("Z0")

In [ ]:
particle.Particle.from_string("Z0").mass

In [ ]:
zGeV = particle.Particle.from_string("Z0").mass / 1000

Another aside, [np.minimum](https://numpy.org/doc/stable/reference/generated/numpy.minimum.html) is an array-at-a-time function to find element-by-element minima between two arrays.

In [ ]:
np.minimum(np.array([1, 2, 3, 4, 5]), np.array([5, 4, 3, 2, 1]))

We can use that to find the closest-to-on-shell Z in each of the two scenarios: 0011 (`z00` and `z11`) or 0110 (`z01` and `z10`).

In [ ]:
zdist_0011 = np.minimum(abs(z00.mass - zGeV), abs(z11.mass - zGeV))
zdist_0110 = np.minimum(abs(z01.mass - zGeV), abs(z10.mass - zGeV))
zdist_0011, zdist_0110

For each event `i`, either `zdist_0011[i]` is nearly zero because it contains a real on-shell Z and a real off-shell Z or `zdist_0110[i]` is.

For each `i`, the wrong case has two crossed muon pairs, not correctly identified Zs, which are both far from 91 GeV.

In [ ]:
is_0011 = zdist_0011 < zdist_0110
is_0011

With an array of booleans, we can pick the right pairing element-by-element using [ak.where](https://awkward-array.readthedocs.io/en/latest/_auto/ak.where.html).

Try negating the booleans with `~`.

In [ ]:
hist.Hist.new.Reg(100, 0, 100, name="distance from 91 GeV").Double().fill(
    ak.where(is_0011, zdist_0011, zdist_0110)
)

Now we'd like to select _objects_ with `is_0011`, rather than numbers, so we first have to build the right objects.

Each element needs to be two Z bosons, an array of length-2 lists of Lorentz vectors.

We can construct this individually for the array of 0011 cases and the array of 0110 cases.

`np.newaxis` unflattens an array by putting each element in a length-1 nested list. (See the [slicing documentation](https://awkward-array.readthedocs.io/en/latest/_auto/ak.Array.html#ak-array-getitem).)

In [ ]:
z00.type

In [ ]:
z00[:, np.newaxis].type

And we can [ak.concatenate](https://awkward-array.readthedocs.io/en/latest/_auto/ak.concatenate.html) two such things at `axis=1` to get length-2 lists.

In [ ]:
z0011_pair = ak.concatenate((z00[:, np.newaxis], z11[:, np.newaxis]), axis=1)
z0110_pair = ak.concatenate((z01[:, np.newaxis], z10[:, np.newaxis]), axis=1)
z0011_pair, z0110_pair

In [ ]:
ak.num(z0011_pair), ak.num(z0110_pair)

Now apply [ak.where](https://awkward-array.readthedocs.io/en/latest/_auto/ak.where.html) list by list to pick the right pair of Zs for each event.

In [ ]:
correct_pair = ak.where(is_0011, z0011_pair, z0110_pair)
correct_pair

In [ ]:
ak.num(correct_pair)

In [ ]:
hist.Hist.new.Reg(120, 0, 120, name="mass").Double().fill(
    correct_pair[:, 0].mass
)

For completeness, look at the wrong pairs:

In [ ]:
hist.Hist.new.Reg(120, 0, 120, name="mass").Double().fill(
    ak.where(~is_0011, z0011_pair, z0110_pair)[:, 0].mass
)

Each of these length-2 lists has a correctly reconstructed on-shell Z and a correctly reconstructed off-shell Z, but in no particular order.

We can sort them by their distance to 91 GeV.

In [ ]:
sort_index = ak.argsort(abs(correct_pair.mass - zGeV), axis=1)
sort_index

In [ ]:
sorted_pair = correct_pair[sort_index]
sorted_pair

In [ ]:
hist.Hist.new.Reg(120, 0, 120, name="mass").Double().fill(
    sorted_pair[:, 0].mass
)

Adding the Lorentz vectors of both Zs gives us the Higgs mass (though we didn't have to correctly identify them to do this):

In [ ]:
hist.Hist.new.Reg(150, 0, 150, name="mass").Double().fill(
    (sorted_pair[:, 0] + sorted_pair[:, 1]).mass
)

<br><br><br><br><br>

## Identifying Z bosons in H → ZZ → 4μ with Numba

Now let's do the same thing with Numba, starting with the hard part: pairing muons into right and wrong combinations and using proximity to 91 GeV of _one_ member of each pair of Z candidates to identify the right combination.

In [ ]:
four_muons

In [ ]:
ak.num(four_muons)

In [ ]:
ak.sum(four_muons.charge, axis=-1)

Pre-sorting the muons with an array-at-a-time function makes it easier to write the Numba-compiled function.

In [ ]:
sorted_four_muons = four_muons[ak.argsort(four_muons.charge, axis=1)]
sorted_four_muons.charge

The logic for picking the right pair is similar to the array-at-a-time case, but instead of "`z00[i]`" being a single Z candidate with `z00` being an array over all events, `z00` is a single Z candidate in a for loop over events.

In this example, we use the [ak.ArrayBuilder](https://awkward-array.readthedocs.io/en/latest/_auto/ak.ArrayBuilder.html) to make the output vector records directly. Just be sure to name them `"Momentum4D"`, so that they are recognized as vectors and not generic records.

(TODO: it would be nice if ArrayBuilder's `append` would take Lorentz vector objects without having to rebuild them...)

In [ ]:
@nb.jit
def make_z_pairs(sorted_four_muons, builder):
    for event in sorted_four_muons:
        # unpack the sorted four muons into appropriately named variables
        mum0, mum1, mup0, mup1 = event

        # either z00 and z11 are correct or z01 and z10 are correct
        z00 = mup0 + mum0
        z11 = mup1 + mum1
        
        z01 = mup0 + mum1
        z10 = mup1 + mum0
        
        # the correct pair of Zs is the pair that has one Z close to 91 GeV
        zdist_0011 = min(abs(z00.mass - zGeV), abs(z11.mass - zGeV))
        zdist_0110 = min(abs(z01.mass - zGeV), abs(z10.mass - zGeV))

        if zdist_0011 < zdist_0110:
            # z00 and z11 are correct
            builder.begin_list()
            builder.begin_record("Momentum4D")
            builder.field("px"); builder.append(z00.px)
            builder.field("py"); builder.append(z00.py)
            builder.field("pz"); builder.append(z00.pz)
            builder.field("E"); builder.append(z00.E)
            builder.end_record()
            builder.begin_record("Momentum4D")
            builder.field("px"); builder.append(z11.px)
            builder.field("py"); builder.append(z11.py)
            builder.field("pz"); builder.append(z11.pz)
            builder.field("E"); builder.append(z11.E)
            builder.end_record()
            builder.end_list()

        else:
            # z01 and z10 are correct
            builder.begin_list()
            builder.begin_record("Momentum4D")
            builder.field("px"); builder.append(z01.px)
            builder.field("py"); builder.append(z01.py)
            builder.field("pz"); builder.append(z01.pz)
            builder.field("E"); builder.append(z01.E)
            builder.end_record()
            builder.begin_record("Momentum4D")
            builder.field("px"); builder.append(z10.px)
            builder.field("py"); builder.append(z10.py)
            builder.field("pz"); builder.append(z10.pz)
            builder.field("E"); builder.append(z10.E)
            builder.end_record()
            builder.end_list()

    return builder

correct_pair = make_z_pairs(sorted_four_muons, ak.ArrayBuilder()).snapshot()
correct_pair

In [ ]:
hist.Hist.new.Reg(120, 0, 120, name="mass").Double().fill(
    correct_pair[:, 0].mass
)

Try inverting the selection to make the `correct_pair` contain the incorrect pair!

In [ ]:
hist.Hist.new.Reg(150, 0, 150, name="mass").Double().fill(
    (correct_pair[:, 0] + correct_pair[:, 1]).mass
)